In [10]:
# print("Tweet 1: ", df['text'][1])
# print("Indice of '{}': {}".format(df['text_token'][1][0], w2v_indices[df['text_token'][1][0]]))
# print("Indice of '{}': {}".format(df['text_token'][1][0], w2v_vectors[w2v_indices[df['text_token'][1][0]]]))
# print("Indice of '{}': {}".format(df['text_token'][1][1], w2v_indices[df['text_token'][1][1]]))
# print("Indice of '{}': {}".format(df['text_token'][1][1], w2v_vectors[w2v_indices[df['text_token'][1][1]]]))
# print("\nVector of the first headline:\n", vectorize(df['text_token'][1]))

In [11]:
# df_train_X[['hasURL','urls_expanded']][(df_train_X["urls_expanded"] == 0) & (df_train_X["hasURL"] != 0)]

---

In [12]:
import pandas as pd
import numpy as np
from glob2 import glob
import json

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stp
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from datetime import datetime
from datetime import date
from datetime import timedelta

[nltk_data] Downloading package punkt to /Users/june/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/june/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)
# pd.set_option('display.max_rowwidth', 100)
pd.set_option('display.max_colwidth', 400)

# TOC
1. Data Import
    1. Feature Extraction
2. Text Vectorization
    2. Word2Vec
    3. Doc2Vec
4. Data Export as CSV files

# Data Import

## Functions

In [14]:
import random


def cross_val_jsons(jsonFiles, isTest):
    # output = [json for json in jsonFiles]
    # print(random.choice(jsonFiles))
    if (isTest == True):
        test = random.choice(jsonFiles)
        jsonFiles.remove(test)
        train = [json for event in jsonFiles for json in event]
        return train, test
    else:
        data = [json for event in jsonFiles for json in event]
        return [data]

def extract_data(datas):
    data_lists = []
    isRumorLists = []
    for index, dataset in enumerate(datas):
        data_list = []
        isRumorList = []
        count = 0 # help var

        for jsonFile in dataset:
            count+=1
            if jsonFile.find("non-rumours") == -1:
                isRumorList.append(1)
            else:
                isRumorList.append(0)

            with open (jsonFile, 'r') as f:
                for l in f.readlines():
                    if not l.strip (): # skip empty lines
                        continue
                    json_data = json.loads(l)
                    # print (json_data,"\n\n")
                    data_list.append(json_data)

        isRumorLists.append(pd.DataFrame(isRumorList, columns=['isRumor']))
        data_lists.append(data_list)
    return data_lists, isRumorLists

def printRumor(route):
    if route.find("rumours") == -1:
        print('non-rumors')
    else:
        print('rumor')

In [15]:
def capitalratio(tweet_text):
    uppers = [l for l in tweet_text if l.isupper()]
    capitalratio = len(uppers) / len(tweet_text)
    return capitalratio 

def tweets2tokens(tweet_text):
    # Tokenizing
    urls = []
    tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', tweet_text.lower()))
    tweet_text = re.sub(r"http\S+", "", tweet_text)
    # tokens = nltk.TweetTokenizer().tokenize(re.sub(r'([\d,.]+)','', tweet_text.lower()))

    # Setting url value (whether the tweet contains http link) and filter http links
    url=0
    for token in tokens:
        if token.startswith('http'):
            url=1
    tokens = [token for token in tokens if not token.startswith('http')]

    ## Stemming
    # porter = PorterStemmer()
    # tokens = [porter.stem(token) for token in tokens]

    # Filtering Stop words
    # from nltk.corpus import stopwords
    # stop_words = set(stopwords.words('english'))
    # tokens = [token for token in tokens if not token in stop_words]

    return tokens, url

def getposcount(tokens):
    postag = []
    poscount = {}
    poscount['Noun']=0
    poscount['Verb']=0
    poscount['Adjective'] = 0
    poscount['Pronoun']=0
    poscount['FirstPersonPronoun']=0
    poscount['SecondPersonPronoun']=0
    poscount['ThirdPersonPronoun']=0
    poscount['Adverb']=0
    poscount['Numeral']=0
    poscount['Conjunction_inj']=0
    poscount['Particle']=0
    poscount['Determiner']=0
    poscount['Modal']=0
    poscount['Whs']=0
    Nouns = {'NN','NNS','NNP','NNPS'}
    Adverbs = {'RB','RBR','RBS'}
    Whs = {'WDT','WP','WRB'} # Composition of wh-determiner(that,what), wh-pronoun(who), wh-adverb(how)
    Verbs={'VB','VBP','VBZ','VBN','VBG','VBD','To'}
    first_person_pronouns=['i','I','me','my','mine','we','us','our','ours'] #'i',
    second_person_pronouns=['you','your','yours']
    third_person_pronouns=['he','she','it','him','her','it','his','hers','its','they','them','their','theirs']

    for word in tokens:
        w_lower=word.lower()
        if w_lower in first_person_pronouns:
            poscount['FirstPersonPronoun']+=1
        elif w_lower in second_person_pronouns:
            poscount['SecondPersonPronoun']+=1
        elif w_lower in third_person_pronouns:
            poscount['ThirdPersonPronoun']+=1
    
    postag = nltk.pos_tag(tokens)
    for g1 in postag:
        if g1[1] in Nouns:
            poscount['Noun'] += 1
        elif g1[1] in Verbs:
            poscount['Verb']+= 1
        elif g1[1]=='ADJ'or g1[1]=='JJ':
            poscount['Adjective']+=1
        elif g1[1]=='PRP' or g1[1]=='PRON' or g1[1]=='PRP$':
            poscount['Pronoun']+=1
        elif g1[1] in Adverbs or g1[1]=='ADV':
            poscount['Adverb']+=1
        elif g1[1]=='CD':
            poscount['Numeral']+=1
        elif g1[1]=='CC' or g1[1]=='IN':
            poscount['Conjunction_inj']+=1
        elif g1[1]=='RP':
            poscount['Particle']+=1
        elif g1[1]=='MD':
            poscount['Modal']+=1
        elif g1[1]=='DT':
            poscount['Determiner']+=1
        elif g1[1] in Whs:
            poscount['Whs']+=1
    return poscount

def contentlength(words):
    wordcount = len(words)
    return wordcount

def extract_urls(entities_dicts):
    if len(entities_dicts) < 1:
        return 0,[],[]

    urls = []
    urls_expanded = []

    key = 'url'
    key2 = 'expanded_url'
    # print(len(entities_dict))
    for i in entities_dicts:
        urls.append(i[key])
        urls_expanded.append(i[key2])
    return 1, urls, urls_expanded

def flatten_tweets(tweets):
    """ Flattens out tweet dictionaries so relevant JSON is in a top-level dictionary. """
    tweets_list = []
    total_tokens_l = []

    # Iterate through each tweet
    for tweet_obj in tweets:
        output_f = dict()

        output_f['text']= tweet_obj['text']
        
        urls_dicts = tweet_obj['entities']['urls']
        # print(urls_dicts)

        output_f['hasURL'], output_f['urls'], output_f['urls_expanded'] = extract_urls(urls_dicts)
        
        # print(type(tweet_obj['user']))
        # print(tweet_obj['user'].contains_key('entities'))
        if ('url' in tweet_obj['user']):
            output_f['hasUserURL'] = 1
            output_f['user_url'] = tweet_obj['user']['url']
        elif ('entities' in tweet_obj['user']):
            # output_f['user_entity'] = tweet_obj['user']['entities']['url']['urls']
            # print(tweet_obj['user']['entities']['url']['urls'])
            # output_f['user_url'] = tweet_obj['user']['entities']['expanded_url']
            output_f['hasUserURL'] , _ , output_f['user_url'] = extract_urls(tweet_obj['user']['entities']['url']['urls'])
        else:
            # output_f['user_entity'] = None
            output_f['user_url'] = 0
            output_f['hasUserURL'] = 0

        output_f['text_token'], output_f['isNotOnlyText'] = tweets2tokens(tweet_obj['text'])
        total_tokens_l.extend(output_f['text_token']) # append the tokens to list of total tokens

        '''POS Tagging'''
        pos_dict=getposcount(output_f['text_token'])
        output_f.update(pos_dict)

        output_f['char_count'] = len(output_f['text'])
        output_f['word_count'] = len(output_f['text_token'])

        output_f['has_question'] = "?" in output_f["text"]
        output_f['has_exclaim'] = "!" in output_f["text"]
        output_f['has_period'] = "." in output_f["text"]
    
        ''' User info'''
        # Store the user screen name in 'user-screen_name'
        # output_f['user-screen_name'] = tweet_obj['user']['screen_name']
        
        # Store the user location
        # output_f['user-location'] = tweet_obj['user']['location']

        acc_created = datetime.strptime(tweet_obj['user']['created_at'], '%a %b %d %H:%M:%S %z %Y')
        tweet_created = datetime.strptime(tweet_obj['created_at'], '%a %b %d %H:%M:%S %z %Y')
        age = (tweet_created - acc_created)
        # print(type(timedelta.total_seconds(age)))

        output_f['capital_ratio']=(capitalratio(tweet_obj['text']))

        # features=(capitalratio(data_list[0]['user']))
        output_f['tweet_count'] = np.log10(tweet_obj['user']['statuses_count'])
        output_f['listed_count'] = np.log10(tweet_obj['user']['listed_count'])
        output_f['follow_ratio'] = np.log10(tweet_obj['user']['followers_count'])
        output_f['age'] = int(timedelta.total_seconds(age)/86400)
        output_f['verified'] = tweet_obj['user']['verified']

        tweets_list.append(output_f)

    unk_tokens_l = list(set(total_tokens_l))
    print("Number of total tokens appeared: {}\nNumber of unique tokens appeared: {}\n".format(len(total_tokens_l), len(unk_tokens_l))) # number of tokens and unique tokens

    return tweets_list

In [16]:
class data_loader():

    def getdata(self, root=True, reaction=False, split=True):
        lists = []
        if root == True:
            charliehebdo_jsons = glob('../pheme-rnr-dataset/charliehebdo/**/source-tweet/*.json') 
            ferguson_jsons = glob('../pheme-rnr-dataset/ferguson/**/source-tweet/*.json')
            germanwing_scrash_jsons = glob('../pheme-rnr-dataset/germanwings-crash/**/source-tweet/*.json')
            ottawashooting_jsons = glob('../pheme-rnr-dataset/ottawashooting/**/source-tweet/*.json')
            sydneysiege_jsons = glob('../pheme-rnr-dataset/sydneysiege/**/source-tweet/*.json')
            lists.append([charliehebdo_jsons, ferguson_jsons, germanwing_scrash_jsons, ottawashooting_jsons, sydneysiege_jsons])
        elif reaction == True:
            charliehebdo_reaction = glob('../pheme-rnr-dataset/charliehebdo/**/reactions/*.json') 
            ferguson_reaction = glob('../pheme-rnr-dataset/ferguson/**/reactions/*.json')
            germanwing_scrash_reaction = glob('../pheme-rnr-dataset/germanwings-crash/**/reactions/*.json')
            ottawashooting_reaction = glob('../pheme-rnr-dataset/ottawashooting/**/reactions/*.json')
            sydneysiege_reaction = glob('../pheme-rnr-dataset/sydneysiege/**/reactions/*.json')
            lists.append([charliehebdo_reaction, ferguson_reaction, germanwing_scrash_reaction, ottawashooting_reaction, sydneysiege_reaction])
            
        return lists

## Importing JSON Files and grouping

In [17]:
jsonFiles = data_loader().getdata()
data = cross_val_jsons(jsonFiles[0], isTest = False)
data_lists, isRumorLists = extract_data(data)

data = data_lists[0]
data_y = isRumorLists[0]
print("(Data of Root tweets) Data: {}".format(len(data)))
print("(Data of Root tweets) Label: {}".format(len(data_y)))

(Data of Root tweets) Data: 5802
(Data of Root tweets) Label: 5802


## Flatten and extract basic features

In [18]:
df_data = pd.DataFrame(flatten_tweets(data))

Number of total tokens appeared: 88175
Number of unique tokens appeared: 8154



In [19]:
df_data[['has_question', 'has_exclaim', 'has_period','verified']] = df_data[['has_question', 'has_exclaim', 'has_period','verified']].astype(int)

## Handling inf and NaN value

In [20]:
for dataset in [df_data]:
    dataset['listed_count'].replace([np.inf, -np.inf], np.nan, inplace=True)
    print(dataset['listed_count'].mean())
    print("Before fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))
    dataset['listed_count'].fillna(0,inplace=True)
    print("After fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))

3.1020948474234245
Before fill: Does the dataset contain NaN value? True
After fill: Does the dataset contain NaN value? False


In [21]:
df_data = pd.concat([df_data,data_y],axis=1)
# df_data.to_csv('./data_notembeded.csv', index = False)

### Reaction files

In [ ]:
# reactionFiles = data_loader().getdata(reaction=True,root=False)
# data_rt = cross_val_jsons(reactionFiles[0], isTest = False)
# data_lists, isRumorLists = extract_data(data_rt)

# data_rt = data_lists[0]
# data_rt_y = isRumorLists[0]
# print("(Data of Root tweets) Data: {}".format(len(data_rt)))
# print("(Data of Root tweets) Label: {}".format(len(data_rt_y)))

# df_data_rt = pd.DataFrame(flatten_tweets(data_rt))

# df_data_rt[['has_question', 'has_exclaim', 'has_period','verified']] = df_data_rt[['has_question', 'has_exclaim', 'has_period','verified']].astype(int)

# for dataset in [df_data_rt]:
#     dataset['listed_count'].replace([np.inf, -np.inf], np.nan, inplace=True)
#     print(dataset['listed_count'].mean())
#     print("Before fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))
#     dataset['listed_count'].fillna(0,inplace=True)
#     print("After fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))
# print(data_rt.shape, data_rt_y.shape)
# df_data_rt.to_csv('./data_rt_notembeded.csv', index = False)

# Word2vec

In [12]:
word2vec_ = w2v.Word2Vec(
    df_data['text_token'],
    sg = 1, 
    seed = 1,
    workers = 8,
    size = 300,
    min_count = 5,
    window = 10,
    sample = 1e-3
)

In [13]:
word2vec_.train(df_data['text_token'], total_examples = word2vec_.corpus_count, epochs = word2vec_.iter)

(299798, 440875)

In [75]:
# word2vec_.wv.vectors.shape # vocab size / window size

In [14]:
word2vec_.save('w2v_model_allRoot')

### Load Word Vector model and Vectorize the sentence

In [22]:
word2vec_ = w2v.Word2Vec.load('w2v_model_allRoot')

In [23]:
word_vectors = word2vec_.wv
vocabs = word_vectors.vocab.keys()

In [24]:
w2v_vectors = word_vectors.vectors # here you load vectors for each word in your model
w2v_indices = {word: word_vectors.vocab[word].index for word in word_vectors.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [25]:
def vectorize(line): 
    words = []
    for word in line: # line - iterable, for example list of tokens 
        try:
            w2v_idx = w2v_indices[word]
        except KeyError: # if you does not have a vector for this word in your w2v model, continue 
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)

## Average of Vectors & Previous features

In [26]:
import copy
df_data['text_token_vec'] = copy.deepcopy(df_data['text_token'])

for index, sentence in enumerate(df_data['text_token_vec']):
    df_data['text_token_vec'][index] = vectorize(sentence).mean(axis=0)

pd.DataFrame(df_data['text_token_vec'].values.tolist()).shape
df_data[['text_token','text_token_vec']].head()
df_data_avg = pd.DataFrame(df_data['text_token_vec'].values.tolist()).add_prefix('token_avg') #.join(df)
df_data_avg = df_data.join(df_data_avg).drop('text_token_vec',axis=1)
df_data.drop(['text_token_vec'],axis=1,inplace=True)

In [27]:
df_data_avg.head(2)

,text,hasURL,urls,urls_expanded,hasUserURL,user_url,text_token,isNotOnlyText,Noun,Verb,Adjective,Pronoun,FirstPersonPronoun,SecondPersonPronoun,ThirdPersonPronoun,Adverb,Numeral,Conjunction_inj,Particle,Determiner,Modal,Whs,char_count,word_count,has_question,has_exclaim,has_period,capital_ratio,tweet_count,listed_count,follow_ratio,age,verified,isRumor,token_avg0,token_avg1,token_avg2,token_avg3,token_avg4,token_avg5,token_avg6,token_avg7,token_avg8,token_avg9,token_avg10,token_avg11,token_avg12,token_avg13,token_avg14,token_avg15,token_avg16,token_avg17,token_avg18,token_avg19,token_avg20,token_avg21,token_avg22,token_avg23,token_avg24,token_avg25,token_avg26,token_avg27,token_avg28,token_avg29,token_avg30,token_avg31,token_avg32,token_avg33,token_avg34,token_avg35,token_avg36,token_avg37,token_avg38,token_avg39,token_avg40,token_avg41,token_avg42,token_avg43,token_avg44,token_avg45,token_avg46,token_avg47,token_avg48,token_avg49,token_avg50,token_avg51,token_avg52,token_avg53,token_avg54,token_avg55,token_avg56,token_avg57,token_avg58,token_avg59,token_avg60,token_avg61,token_avg62,token_avg63,token_avg64,token_avg65,token_avg66,token_avg67,token_avg68,token_avg69,token_avg70,token_avg71,token_avg72,token_avg73,token_avg74,token_avg75,token_avg76,token_avg77,token_avg78,token_avg79,token_avg80,token_avg81,token_avg82,token_avg83,token_avg84,token_avg85,token_avg86,token_avg87,token_avg88,token_avg89,token_avg90,token_avg91,token_avg92,token_avg93,token_avg94,token_avg95,token_avg96,token_avg97,token_avg98,token_avg99,token_avg100,token_avg101,token_avg102,token_avg103,token_avg104,token_avg105,token_avg106,token_avg107,token_avg108,token_avg109,token_avg110,token_avg111,token_avg112,token_avg113,token_avg114,token_avg115,token_avg116,token_avg117,token_avg118,token_avg119,token_avg120,token_avg121,token_avg122,token_avg123,token_avg124,token_avg125,token_avg126,token_avg127,token_avg128,token_avg129,token_avg130,token_avg131,token_avg132,token_avg133,token_avg134,token_avg135,token_avg136,token_avg137,token_avg138,token_avg139,token_avg140,token_avg141,token_avg142,token_avg143,token_avg144,token_avg145,token_avg146,token_avg147,token_avg148,token_avg149,token_avg150,token_avg151,token_avg152,token_avg153,token_avg154,token_avg155,token_avg156,token_avg157,token_avg158,token_avg159,token_avg160,token_avg161,token_avg162,token_avg163,token_avg164,token_avg165,token_avg166,token_avg167,token_avg168,token_avg169,token_avg170,token_avg171,token_avg172,token_avg173,token_avg174,token_avg175,token_avg176,token_avg177,token_avg178,token_avg179,token_avg180,token_avg181,token_avg182,token_avg183,token_avg184,token_avg185,token_avg186,token_avg187,token_avg188,token_avg189,token_avg190,token_avg191,token_avg192,token_avg193,token_avg194,token_avg195,token_avg196,token_avg197,token_avg198,token_avg199,token_avg200,token_avg201,token_avg202,token_avg203,token_avg204,token_avg205,token_avg206,token_avg207,token_avg208,token_avg209,token_avg210,token_avg211,token_avg212,token_avg213,token_avg214,token_avg215,token_avg216,token_avg217,token_avg218,token_avg219,token_avg220,token_avg221,token_avg222,token_avg223,token_avg224,token_avg225,token_avg226,token_avg227,token_avg228,token_avg229,token_avg230,token_avg231,token_avg232,token_avg233,token_avg234,token_avg235,token_avg236,token_avg237,token_avg238,token_avg239,token_avg240,token_avg241,token_avg242,token_avg243,token_avg244,token_avg245,token_avg246,token_avg247,token_avg248,token_avg249,token_avg250,token_avg251,token_avg252,token_avg253,token_avg254,token_avg255,token_avg256,token_avg257,token_avg258,token_avg259,token_avg260,token_avg261,token_avg262,token_avg263,token_avg264,token_avg265,token_avg266,token_avg267,token_avg268,token_avg269,token_avg270,token_avg271,token_avg272,token_avg273,token_avg274,token_avg275,token_avg276,token_avg277,token_avg278,token_avg279,token_avg280,token_avg281,token_avg282,token_avg283,token_avg284,token_avg285,token_avg286,token_avg287,token_avg28

# Doc2Vec

## Training

In [28]:
#Doc2vec 실행
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df_data['text_token'])]
Doc2vec_model = Doc2Vec(vector_size=300, min_alpha=0.025, window=10, min_count=1, workers=4, epochs=120) #documents,
Doc2vec_model.build_vocab(documents)

Doc2vec_model.train(documents, epochs=Doc2vec_model.epochs, total_examples=Doc2vec_model.corpus_count)

print("The Dimension of a vector is: {}".format(len(Doc2vec_model.docvecs[0]))) # document vector의 정보

The Dimension of a vector is: 300


In [29]:
def most_similar_text(index):
    similar = Doc2vec_model.docvecs.most_similar(index)

    print("The quried text: \n\n{} \n\nis most similar to the text:\n\n{}".format(df_data['text'][index],df_data['text'][similar[0][0]]))

# most_similar_text(100)

In [30]:
Doc2vec_model.save('./d2v_model_whole')

## Infer the document vectors from trained Doc2Vec Model

In [31]:
Doc2vec_model = Doc2Vec.load('./d2v_model_whole')

### Train data

In [32]:
df_data['text_token_doc'] = copy.deepcopy(df_data['text_token'])

for index, sentence in enumerate(df_data['text_token_doc']):
    df_data['text_token_doc'][index] = Doc2vec_model.infer_vector(df_data['text_token_doc'][index],steps=50)

df_data_doc = pd.DataFrame(df_data['text_token_doc'].values.tolist()).add_prefix('doc_vec')
df_data_doc = df_data.join(df_data_doc).drop('text_token_doc',axis=1)

# Exporting data

In [53]:
'''Doc2vec Vector Feature set'''
df_data_avg.to_csv('./data_avg.csv', index = False)
df_data_doc.to_csv('./data_doc.csv', index = False)

# Validation data

In [ ]:
import copy

In [114]:
gurlitt_jsons = glob('../PHEME/all-rnr-annotated-threads/gurlitt-all-rnr-threads/**/source-tweets/*.json')
ebolaessien_jsons = glob('../PHEME/all-rnr-annotated-threads/ebola-essien-all-rnr-threads/**/source-tweets/*.json')
putinmissing_jsons = glob('../PHEME/all-rnr-annotated-threads/putinmissing-all-rnr-threads/**/source-tweets/*.json')

added_files = [gurlitt_jsons, ebolaessien_jsons, putinmissing_jsons]

valid_data = cross_val_jsons(added_files, False)
data_lists, isRumorLists = extract_data(valid_data)
X_valid = data_lists[0]
y_valid = isRumorLists[0]
print("(Data of Root tweets) X: {} y: {}".format(len(X_valid),len(y_valid)))

df_valid_X = pd.DataFrame(flatten_tweets(X_valid))
df_valid = pd.concat([df_valid_X,y_valid],axis=1)
df_valid[['has_question', 'has_exclaim', 'has_period','verified']] = df_valid[['has_question', 'has_exclaim', 'has_period','verified']].astype(int)

for dataset in [df_valid]:
    dataset['listed_count'].replace([np.inf, -np.inf], np.nan, inplace=True)
    # print(dataset['listed_count'].mean())
    print("Before fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))
    dataset['listed_count'].fillna(0,inplace=True)
    print("After fill: Does the dataset contain NaN value? {}".format(np.any(np.isnan(dataset['listed_count']))))
print("\nDropping tweets with short length (<10)....\n",df_valid.shape)
df_valid.to_csv('./data_valid_notembeded.csv', index = False)

df_valid.drop(df_valid[df_valid['word_count'] < 10].index, inplace=True)
df_valid.reset_index(drop=True, inplace=True)
print(" ->",df_valid.shape)

(Data of Root tweets) X: 390 y: 390
Number of total tokens appeared: 5637
Number of unique tokens appeared: 1516

Before fill: Does the dataset contain NaN value? True
After fill: Does the dataset contain NaN value? False

Dropping tweets with short length (<10)....
 (390, 34)
 -> (328, 34)


In [106]:
word2vec_ = w2v.Word2Vec.load('w2v_model_whole')
word_vectors = word2vec_.wv
w2v_vectors = word_vectors.vectors # here you load vectors for each word in your model
w2v_indices = {word: word_vectors.vocab[word].index for word in word_vectors.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

df_valid['text_token_vec'] = copy.deepcopy(df_valid['text_token'])

for index, sentence in enumerate(df_valid['text_token_vec']):
    df_valid['text_token_vec'][index] = vectorize(sentence).mean(axis=0)

avg = pd.DataFrame(df_valid['text_token_vec'].values.tolist()).add_prefix('token_avg')
df_valid_avg = df_valid.join(avg).drop('text_token_vec',axis=1)

df_valid.drop(['text_token_vec'],axis=1,inplace=True)

In [110]:
Doc2vec_model = Doc2Vec.load('./d2v_model_whole')
df_valid['text_token_doc'] = copy.deepcopy(df_valid['text_token'])

for index, sentence in enumerate(df_valid['text_token_doc']):
    df_valid['text_token_doc'][index] = Doc2vec_model.infer_vector(df_valid['text_token_doc'][index],steps=50)

doc = pd.DataFrame(df_valid['text_token_doc'].values.tolist()).add_prefix('doc_vec')
df_valid_doc = df_valid.join(doc).drop('text_token_doc',axis=1)

df_valid.drop(['text_token_doc'],axis=1,inplace=True)

In [112]:
df_valid_avg.to_csv('./valid_avg.csv', index = False)
df_valid_doc.to_csv('./valid_doc.csv', index = False)